In [1]:
import os

import numpy as np
import pandas as pd
from astropy.coordinates import EarthLocation

import pyuvdata.utils as uvutils

mwa_site = EarthLocation.of_site("mwa")
mwa_site_ecef = np.array([mwa_site.x.value, mwa_site.y.value, mwa_site.z.value])

In [2]:
phase1_enu_file = "mwa_phase1_antpos_enu.csv"
phase2_enu_file = "mwa_phase2_antpos_enu.csv"

In [3]:
dtype_phase1 = np.dtype([("antenna_number",int),("height", float), ("east", float), ("north", float)])
phase1_arr = np.genfromtxt(phase1_enu_file, delimiter=",", skip_header=1, dtype=dtype_phase1)

In [4]:
antnums_phase1 = phase1_arr["antenna_number"]
antnames_phase1 = np.array([f"Tile{antname:03d}" for antname in antnums_phase1])

In [5]:
enu_phase1 = np.stack((phase1_arr["east"], phase1_arr["north"], phase1_arr["height"]), axis=1)

In [6]:
dtype_phase2 = np.dtype([("antenna_name",np.unicode_, 16),("height", float), ("east", float), ("north", float)])
phase2_arr = np.genfromtxt(phase2_enu_file, delimiter=",", skip_header=1, dtype=dtype_phase2)

In [7]:
antnames_phase2 = np.array([antname.strip() for antname in phase2_arr["antenna_name"]])
antnums_phase2 = np.concatenate((2000 + np.arange(1,57), 1000 + np.arange(1,73)))

In [8]:
antnames_phase2

array(['LBA1', 'LBA2', 'LBA3', 'LBA4', 'LBA5', 'LBA6', 'LBA7', 'LBA8',
       'LBB1', 'LBB2', 'LBB3', 'LBB4', 'LBB5', 'LBB6', 'LBB7', 'LBB8',
       'LBC1', 'LBC2', 'LBC3', 'LBC4', 'LBC5', 'LBC6', 'LBC7', 'LBC8',
       'LBD1', 'LBD2', 'LBD3', 'LBD4', 'LBD5', 'LBD6', 'LBD7', 'LBD8',
       'LBE1', 'LBE2', 'LBE3', 'LBE4', 'LBE5', 'LBE6', 'LBE7', 'LBE8',
       'LBF1', 'LBF2', 'LBF3', 'LBF4', 'LBF5', 'LBF6', 'LBF7', 'LBF8',
       'LBG1', 'LBG2', 'LBG3', 'LBG4', 'LBG5', 'LBG6', 'LBG7', 'LBG8',
       'HexE01', 'HexE02', 'HexE03', 'HexE04', 'HexE05', 'HexE06',
       'HexE07', 'HexE08', 'HexE09', 'HexE10', 'HexE11', 'HexE12',
       'HexE13', 'HexE14', 'HexE15', 'HexE16', 'HexE17', 'HexE18',
       'HexE19', 'HexE20', 'HexE21', 'HexE22', 'HexE23', 'HexE24',
       'HexE25', 'HexE26', 'HexE27', 'HexE28', 'HexE29', 'HexE30',
       'HexE31', 'HexE32', 'HexE33', 'HexE34', 'HexE35', 'HexE36',
       'HexS01', 'HexS02', 'HexS03', 'HexS04', 'HexS05', 'HexS06',
       'HexS07', 'HexS08', 'HexS09

In [9]:
enu_phase2 = np.stack((phase2_arr["east"], phase2_arr["north"], phase2_arr["height"]), axis=1)

In [10]:
antnames_special = np.array(["AAVS0.5", "BIGHORNS", "EDA1", "RFI_monitor", "CRAM", "EDA2"])
antnums_special = np.array([79, 59, 99, 999, 1099, 2999])
assert antnames_special.size == antnums_special.size
enu_special = np.zeros((antnums_special.size, 3), dtype=float)

In [11]:
antnums_full = np.concatenate((antnums_phase1, antnums_phase2, antnums_special))
antnames_full = np.concatenate((antnames_phase1, antnames_phase2, antnames_special))

In [12]:
enu_full = np.concatenate((enu_phase1, enu_phase2, enu_special), axis=0)

In [13]:
rel_ecef_full = uvutils.ECEF_from_ENU(
    enu_full,
    latitude = mwa_site.geodetic.lat.rad,
    longitude=mwa_site.geodetic.lon.rad,
    altitude=mwa_site.geodetic.height
) - mwa_site_ecef

In [14]:
rel_ecef_full.shape

(262, 3)

In [15]:
ant_pos_df = pd.DataFrame(columns = ["names", "numbers", "x", "y", "z"])
ant_pos_df["names"] = antnames_full
ant_pos_df["numbers"] = antnums_full
ant_pos_df["x"] = rel_ecef_full[:,0]
ant_pos_df["y"] = rel_ecef_full[:,1]
ant_pos_df["z"] = rel_ecef_full[:,2]

In [16]:
ant_pos_df = ant_pos_df.sort_values("numbers")

In [17]:
ant_pos_df.to_csv("mwa_ant_pos.csv", index=False)